# Using a Weighted Overlay Analysis to Determine Optimal Paths for Summitting

### Eric Gibson
### GIS 5571: ArcGIS I
### December 18, 2023

### Abstract

   Summiting is a popular activity for hikers, backpackers, and other outdoor enthusiasts. This study uses 10-meter spatial resolution digital elevation models and land cover raster datasets to determine optimal paths for summiteers using an ArcGIS Pro Notebook. Eleven western United States state high points were chosen to create these optimal routes due to their remoteness and degree of difficulty. These paths were created via a proportional weighting of slope angle and a determination of weighting for land cover based on general hiking preferences. Additionally, weighting was performed based on preference of slope over land cover and vice versa. Water features were also considered, including ponds, lakes (both of which were included within the land cover raster dataset), streams, and rivers (created via flow accumulation values). The paths provided helpful instruction for summiting peaks present within this study, however future analyses will be needed with more accurate and additional data from users for more precise and specified outputs.

## Notebook setup

In [15]:
from arcpy import env
import requests
import glob
import os
import zipfile

Establish variables for the ArcGIS Project, Map, and spatial reference.

In [ ]:
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]
spatial_ref = arcpy.SpatialReference(4326)

In [5]:
folder = r'%s' % input('Enter preferred folder directory: ')
%store folder

print(folder)

Enter preferred folder directory: C:\Users\15612\Documents\GIS-5571\Final Project
Stored 'folder' (str)
C:\Users\15612\Documents\GIS-5571\Final Project


## Gather Data (ETL Pipeline)

Enter coordinates for bounding box of the project:

In [6]:
east = input('Enter x-coordinate of east-most side of bounding box: ')
west = input('Enter x-coordinate of west-most side of bounding box: ')
north = input('Enter y-coordinate of north-most side of bounding box: ')
south = input('Enter y-coordinate of south-most side of bounding box: ')

%store east
%store west
%store north
%store south

Enter x-coordinate of east-most side of bounding box: 45.5
Enter x-coordinate of west-most side of bounding box: -109.56
Enter y-coordinate of north-most side of bounding box: 46.4
Enter y-coordinate of south-most side of bounding box: -106.45
Stored 'east' (str)
Stored 'west' (str)
Stored 'north' (str)
Stored 'south' (str)


In [16]:
b_box = arcpy.Polygon(arcpy.Array(
    [arcpy.Point(float(east),float(north)),
    arcpy.Point(float(west),float(north)),
    arcpy.Point(float(west),float(south)),
    arcpy.Point(float(east),float(south))]
))

arcpy.management.CopyFeatures(
    b_box,
    "Bounding_Box"
)

<Result 'C:\\Users\\15612\\Documents\\ArcGIS\\Projects\\Final Project GIS5571\\Final Project GIS5571.gdb\\Bounding_Box'>

Enter coordinates for your destination:

In [37]:
x_destination = input('Enter x coordinate of desired destination: ')
y_destination = input('Enter y coordinate of desired destination: ')

{'html_attributions': [], 'results': [{'formatted_address': 'Gannett Peak, Wyoming 82513, United States', 'geometry': {'location': {'lat': 43.1843954, 'lng': -109.6540391}, 'viewport': {'northeast': {'lat': 43.19253080000001, 'lng': -109.6380317}, 'southwest': {'lat': 43.17625899999999, 'lng': -109.6700465}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Gannett Peak', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111522605089377864134">A Google User</a>'], 'photo_reference': 'AWU5eFhBwj65DIg_MAcDR7-DmBSOunmZoRaY06XDadDxofCGcn2f4YPtclhsJw1vsGoBxNQU9JfRGMhigFY7feMbh-l_b-FZuS4sgkM08lnhzx4YgbjajcGJhXjvtusFu2SK2THuJN2hgjRGRTf7k0yMwcWkEARL1i3DLelCRoFS1-Kw2Gnw', 'width': 4032}], 'place_id': 'ChIJM3jnbmb5V4cRAxzJ46vPbn8', 'rating': 4.6, 'reference': 'ChIJM3jnbmb5V4cR

Enter coordinates for your starting point:

In [34]:
x_starting_point = input('Enter x coordinate of desired starting point: ')
y_starting_point = input('Enter y coordinate of desired starting point: ')

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': 'Forest Rd 650A, Cora, WY 82925, United States', 'geometry': {'location': {'lat': 43.3107957, 'lng': -109.8575453}, 'viewport': {'northeast': {'lat': 43.31216667989273, 'lng': -109.8562174701073}, 'southwest': {'lat': 43.30946702010728, 'lng': -109.8589171298927}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/park-71.png', 'icon_background_color': '#4DB546', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet', 'name': 'Green River Lakes Recreation Area', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100080302452992505051">Thomas Smith</a>'], 'photo_reference': 'AWU5eFidzdISf7eNB_EHTnoy68jZ7efeM2-jmXx_r7YqhLz_ZcV9lXLFdqCfSI8PKAhIgbzDA5QThHeTitgVudeWW__O0FrlNXXJkyKZbIHrT7SNVWY06k8f0YN27FBA8-6Tdw7ov34cLo0KQsOXikkcUklvPcMNE_SHUDdhD_KoCR3TPMwU', 'widt

Create points for the starting and destination points:

In [39]:
# Create a point for the destination within the ArcGIS Pro map

destination = arcpy.PointGeometry(
    arcpy.Point(float(x_destination),float(y_destination)),
    spatial_reference = spatial_ref
)

arcpy.management.CopyFeatures(
    destination,
    'Destination'
)

<Result 'C:\\Users\\15612\\Documents\\ArcGIS\\Projects\\Final Project GIS5571\\Final Project GIS5571.gdb\\Gannett_Peak'>

In [35]:
# Create a point for the starting point within the ArcGIS Pro map

starting_point = arcpy.PointGeometry(
    arcpy.Point(float(x_starting_point),float(y_starting_point)),
    spatial_reference = spatial_ref
)

arcpy.management.CopyFeatures(
    starting_point,
    'Starting_Point'
)

<Result 'C:\\Users\\15612\\Documents\\ArcGIS\\Projects\\Final Project GIS5571\\Final Project GIS5571.gdb\\Starting_Point'>

## Gather and prepare DEM data for analysis

Intially gather DEM data from the USGS National Map API for the analysis

In [21]:
nat_map_url = input('Enter URL of DEM data from the USGS National Map API: ')

# Use requests.get() function for bringing in GeoJSON data
nat_map_data = requests.get(nat_map_url)
nat_map_json = nat_map_data.json()
print(nat_map_json)

{'total': 2, 'items': [{'title': 'USGS 1/3 Arc Second n43w110 20220805', 'moreInfo': 'This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as [...]', 'sourceId': '62edf5d5d34eacf539725807', 'sourceName': 'ScienceBase', 'sourceOriginId': None, 'sourceOriginName': 'gda', 'metaUrl': 'https://www.sciencebase.gov/cata

In [ ]:
# Create a function to automatically write gathered data into a local folder

def write_to_file(obj,file_name):
    
    with open(file_name,'wb') as file:
        file.write(obj.content)

In [22]:
# Loop through each item in GeoJSON and use requests.get() function to acquire the data

count = 0
for item in nat_map_json:
    count += 1
    DEM = requests.get(item['downloadURL'])
    
    write_to_file(DEM,folder + '\DEM\DEM_' + str(count) + '.tif')
    m.addDataFromPath(DEM,folder + '\DEM\DEM_' + str(count) + '.tif')

In [ ]:
# Loop through each file in DEM folder path and add all .tif files to a list

file_list = glob.glob(os.path.join(folder + '\DEM','*'))
DEM_file_list = []
for file in file_list:
    if file[-4:] == ".tif":
        DEM_file_list.append(file)
        
print(DEM_file_list)

Merge gathered DEMs together (if needed) and clip using the created Bounding_Box feature class

In [ ]:
# Merge DEMs in list together

stitched_DEMs = arcpy.ia.Merge(DEM_file_list)

In [ ]:
# Clip DEMs to area of study

clipped_DEM = arcpy.management.Clip(
    in_raster = stitched_DEMs,
    rectangle = 'Bounding_Box',
    out_raster = "clipped_DEM"
)

## Gather and prepare trail data for analysis

Initially gather trail data to use in analysis

In [23]:
# Enter trail data URL and use requests.get() function to gather data

trails_url = input('Enter download URL of preferred trail data: ')

transportation = requests.get(trails_url)

In [25]:
# Use created "write_to_file" function to write the gathered transportation data to a zipfile

write_to_file(transportation,folder + '\Trails\transport.zip')

In [26]:
# Extract all zipfile data to folder

with zipfile.ZipFile(folder + '\Trails\transport.zip') as transport:
    transport.extractall(folder + '\Trails\transport')

In [27]:
# Enter the name of the trails file to use in analysis

trails_name = input('Enter name of the trails dataset to be used: ')

m.addDataFromPath(folder + '\\Trails\\transport\\' + trails_name)

In [ ]:
# Project trails feature to WGS 84

arcpy.management.Project(
        in_dataset = trails_name,
        out_dataset = trails_name + '_project',
        out_coor_system = 4326
    )

In [ ]:
# Add WGS 84 projection to the Bounding_Box feature

arcpy.management.DefineProjection(
    in_dataset = 'Bounding_Box',
    coor_system = spatial_ref
)

Clip the trail data to the extent of the analysis

In [ ]:
# Clip polyline trail data to size of the Bounding_Box

vec_trail_clip = arcpy.analysis.Clip(
    in_features = 'Trans_TrailSegment_project',
    clip_features = 'Bounding_Box',
    out_feature_class = 'vec_trail_clip'
)

## Gather and prepare land cover data for analysis

In [33]:
# Enter URL for land cover land use Sentinel-2 data

land_use_url = input('Enter download URL for Sentinel-2 land cover land use data from the ESRI Living Atlas: ')

land_use = requests.get(
    land_use_url,
    verify = False
)

write_to_file(land_use,folder + '\Land Cover\land_cover.tif')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lulctimeseries.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [35]:
# Add land cover data to ArcGIS Pro map

m.addDataFromPath(folder + '\Land Cover\land_cover.tif')

In [ ]:
# Make cell size for raster analyses equal to a 10-meter spatial resolution

arcpy.env.cellSize = folder + '\Land Cover\land_cover.tif'

Clip the land cover data to the extent of the analysis

In [36]:
# Clip land cover to Bounding_Box feature

clipped_land_cover = arcpy.management.Clip(
    in_raster = folder + '\Land Cover\land_cover.tif',
    rectangle = "Bounding_Box",
    out_raster = "clipped_land_cover"
)

# Important: Please run "Streams, Rivers, and Trails" notebook here

Due to issues with the ArcGIS Pro raster calculator, the extent within the environment will cause an unresolved error to occur. Due to this, please move to running the "Streams, Rivers, and Trails" notebook for this section of the analysis. Once that notebook is run, move back to this notebook and continue at "Begin creating cost surfaces".

## Begin creating cost surfaces

Create the original slope raster

In [1]:
# Create a slope raster using the DEM and ensure the values are in angle degrees

slope_raster = arcpy.sa.Slope(
    clipped_DEM,
    "DEGREE"
)

NameError: name 'arcpy' is not defined

#### Weight the land cover raster based on land cover types:

Open water = "NODATA"

Forest = 3

Farmland and Flooded Vegetation = 4

Snow and Ice = 9

All other land cover types = 2

In [41]:
# Use Reclassify tool to weight land cover types based on difficulty to traverse by foot

weighted_land_use_no_water = arcpy.sa.Reclassify(
    "clipped_land_cover",
    "Value",
    arcpy.sa.RemapValue([[0,0,2],[1,1,'NODATA'],[2,2,3],[3,3,2],[4,4,4],[5,5,4],[6,6,2],[7,8,2],[9,9,9],[10,10,2],[11,255,2]])
)

## Combine all rasters together

Combine the raster defining rivers and streams with the created weighted land use raster dataset

In [56]:
# Combine the rivers_and_streams raster with the weighted_land_use_no_water raster using a conditional statement

updated_land_use = arcpy.sa.Con(
    in_conditional_raster = 'rivers_and_streams',
    in_true_raster_or_constant = 'weighted_land_use_no_water',
    in_false_raster_or_constant = 'rivers_and_streams',
    where_clause = '"Value" = 1'
)

In [57]:
# Create a raster equal to 1 outside of "NODATA" areas to clean the updated_land_use raster

water_no_data = arcpy.sa.Reclassify(
    in_raster = 'weighted_land_use_no_water',
    reclass_field = 'Value',
    remap = arcpy.sa.RemapRange([[1,9,1]])
)

In [58]:
# Multiply the updated_land_use and water_no_data rasters together

final_land_cover = arcpy.sa.RasterCalculator(
    ['updated_land_use','water_no_data'],
    ['x','y'],
    'x * y'
)


# Add raster to map

final_land_cover.save(folder + '\Final Land Cover\final_land_cover.tif')
m.addDataFromPath(folder + '\Final Land Cover\final_land_cover.tif')

Combine the trails_reclass raster with the final_land_cover.tif raster

In [59]:
# Use a conditional statement to combine trail data with the land cover data

final_land_trails = arcpy.sa.Con(
    in_conditional_raster = 'trails_reclass',
    in_true_raster_or_constant = 'final_land_cover.tif',
    in_false_raster_or_constant = 'trails_reclass',
    where_clause = '"Value" = 0'
)

Combine the trails_reclass raster with the slope_raster raster

This will assign a value of 1 to the trails raster, the lowest value within the final cost surface. This allows the optimal path to follow trails if they are present.  

In [64]:
# Use a conditional statement to combine trail data with the slope data

slope_trails = arcpy.sa.Con(
    in_conditional_raster = 'trails_reclass',
    in_true_raster_or_constant = 'slope_raster',
    in_false_raster_or_constant = 'trails_reclass',
    where_clause = '"Value" = 0'
)

## Create final cost surfaces and optimal paths

Create individual preference cost surfaces

In [10]:
# Create for loop from a list containing values of 0.1 to 0.9 in increments of 0.1

for num in range(1,10):
    slope_weight = round(num*0.1, 1)
    land_cover_weight = round(1-num*0.1, 1)
    
    # Multiply slope_trails and final_land_trails by the weight multipliers 
    slope_rast = arcpy.sa.Raster('slope_trails') * slope_weight
    land_cover_rast = arcpy.sa.Raster('final_land_trails') * land_cover_weight
    
    # Add these created rasters to ArcGIS Pro map
    slope_rast.save(folder + '\Slopes\weighted_slope_' + str(num) + '.tif')
    land_cover_rast.save(folder + '\Land Uses\weighted_land_' + str(num) + '.tif')
    
    m.addDataFromPath(folder + '\Slopes\weighted_slope_' + str(num) + '.tif')
    m.addDataFromPath(folder + '\Land Uses\weighted_land_' + str(num) + '.tif')
    
    # Add each of these created rasters together to form a series of cost surfaces
    cs = arcpy.sa.RasterCalculator(
            rasters = ['weighted_slope_' + str(num) + '.tif',
             'weighted_land_' + str(num) + '.tif'],
            input_names = ['x','y'],
            expression = 'x + y'
        )
    
    # Add cost surfaces to ArcGIS Pro map
    cs.save(folder + '\Cost Surfaces\CS_' + str(num) + '.tif')  
    m.addDataFromPath(folder + '\Cost Surfaces\CS_' + str(num) + '.tif')

RuntimeError: Failed to apply Raster Function: 'RasterCalculator' (The parameter is incorrect. 
Parameter 'Rasters' is missing or invalid. Bind failed in function 'Raster Calculator Function' [Raster Calculator Function].)

Create main cost surface

In [65]:
# Create the main cost surface by simply multiplying both the final_land_trails and slope_trails rasters together

cost_surface = arcpy.sa.RasterCalculator(
    ['final_land_trails','slope_trails'],
    ['x','y'],
    'x * y'
)

cost_surface.save(folder + '\Cost Surfaces\Main_CS.tif')
m.addDataFromPath(folder + '\Cost Surfaces\Main_CS.tif')

Create individual preference optimal paths

In [7]:
# Create for loop to calculate the optimal paths for each individual preference cost surface

for num in range(1,10):
    
    # Create distance accumulation raster using the starting point and cost surface while outputting the back direction raster for each cost surface
    dist_accu = arcpy.sa.DistanceAccumulation(
        in_source_data = "Starting_Point",
        in_cost_raster = 'CS_' + str(num) + '.tif',
        out_back_direction_raster = 'back_dir_' + str(num),
        source_direction = 'FROM_SOURCE'
    )
    
    # Create optimal paths for each individual preference using the destination, distance accumulation, and back direction rasters
    optimal_path = arcpy.sa.OptimalPathAsLine(
        in_destination_data = "Destination",
        in_distance_accumulation_raster = dist_accu,
        in_back_direction_raster = 'back_dir_' + str(num),
        out_polyline_features = 'Optimal_Path_' + str(num),
        destination_field = 'OBJECTID',
        path_type = "BEST_SINGLE"
    )

Create main optimal path

In [36]:
# Create main distance accumulation raster using the starting point and main cost surface while outputting the main back direction raster
dist_accu = arcpy.sa.DistanceAccumulation(
        in_source_data = "Starting_Point",
        in_cost_raster = 'Main_CS.tif',
        out_back_direction_raster = 'back_dir',
        source_direction = 'FROM_SOURCE'
    )

# Create main optimal path using the destination, distance accumulation, and main back direction raster
optimal_path = arcpy.sa.OptimalPathAsLine(
        in_destination_data = "Destination",
        in_distance_accumulation_raster = dist_accu,
        in_back_direction_raster = 'back_dir',
        out_polyline_features = 'Main_Optimal_Path',
        destination_field = 'OBJECTID',
        path_type = "EACH_CELL"
    )